In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

# 데이터로드

In [2]:
train_path = "../modified_data/v03_train.csv"
test_path = "../modified_data/v03_test.csv"

In [3]:
train = pd.read_csv(train_path, index_col=0)
test = pd.read_csv(test_path,index_col=0)

In [4]:
train["is_test"] = 0
test["is_test"] = 1
concat = pd.concat([train, test])

In [5]:
column_date = ["k-사용검사일-사용승인일","k-등록일자", "k-수정일자", "단지승인일", "단지신청일"]

for column in column_date:
    print(column, concat[column].dtypes)
    concat[column] = pd.to_datetime(concat[column])
    print(concat[column].dtypes)

k-사용검사일-사용승인일 object
datetime64[ns]
k-등록일자 object
datetime64[ns]
k-수정일자 object
datetime64[ns]
단지승인일 object
datetime64[ns]
단지신청일 object
datetime64[ns]


In [6]:
continuous_columns = []
categorical_columns = []
for column in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[column]) or pd.api.types.is_datetime64_ns_dtype(concat[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '등기신청일자', 'k-전체동수', 'k-전체세대수', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-등록일자', 'k-수정일자', '건축면적', '주차대수', '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', 'target', 'is_test']
범주형 변수: ['시군구', '번지', '아파트명', '거래유형', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', '기타/의무/임대/임의=1/2/3/4']


# 인코딩 전에 어떤 범주형 변수에 어떤 값이 들어있는지 확인

In [7]:
for column in categorical_columns:
    print(column)
    display(concat[column].unique())

시군구


array(['서울특별시 강남구 개포동', '서울특별시 강남구 논현동', '서울특별시 강남구 대치동', '서울특별시 강남구 도곡동',
       '서울특별시 강남구 삼성동', '서울특별시 강남구 세곡동', '서울특별시 강남구 수서동', '서울특별시 강남구 신사동',
       '서울특별시 강남구 역삼동', '서울특별시 강남구 압구정동', '서울특별시 강남구 일원동',
       '서울특별시 강남구 자곡동', '서울특별시 강남구 청담동', '서울특별시 강동구 강일동', '서울특별시 강남구 율현동',
       '서울특별시 강동구 고덕동', '서울특별시 강동구 길동', '서울특별시 강동구 둔촌동', '서울특별시 강동구 명일동',
       '서울특별시 강동구 상일동', '서울특별시 강동구 성내동', '서울특별시 강동구 암사동', '서울특별시 강동구 천호동',
       '서울특별시 강북구 미아동', '서울특별시 강북구 번동', '서울특별시 강서구 가양동', '서울특별시 강북구 수유동',
       '서울특별시 강북구 우이동', '서울특별시 강서구 내발산동', '서울특별시 강서구 등촌동',
       '서울특별시 강서구 공항동', '서울특별시 강서구 마곡동', '서울특별시 강서구 방화동', '서울특별시 강서구 염창동',
       '서울특별시 강서구 화곡동', '서울특별시 관악구 남현동', '서울특별시 관악구 봉천동', '서울특별시 관악구 신림동',
       '서울특별시 광진구 광장동', '서울특별시 광진구 구의동', '서울특별시 광진구 자양동', '서울특별시 광진구 군자동',
       '서울특별시 광진구 화양동', '서울특별시 구로구 개봉동', '서울특별시 구로구 고척동', '서울특별시 광진구 중곡동',
       '서울특별시 구로구 가리봉동', '서울특별시 구로구 구로동', '서울특별시 구로구 오류동',
       '서울특별시 구로구 천왕동', '서울특별시 금천구 독산동', '서울특별시 구로구 온수동', '서울특별시 구로구 궁동',
 

번지


array(['658-1', '652', '12-2', ..., '253-89', '67-5', '240'], dtype=object)

아파트명


...

array(['개별난방', '지역난방', '기타', nan, '중앙난방'], dtype=object)

k-건설사(시공사)


array(['우성건설', '박연정', '대한주택공사', '현대건설(주)', '동양고속건설주식회사', '대우건설',
       '대림산업(주)', '삼성물산', nan, '삼성물산주식회사', '현대,쌍용,GS', '쌍용건설(주)',
       '현대산업개발', '한보', '삼성중공업', '대성산업', '중앙건설', '세방기업', '태영건설', '신동아',
       '신동아건설', '고려개발(주)', '대우건설,대림산업,태영건설', '한라, 두진', '고려산업개발(주)',
       '삼성건설', '롯데건설', 'GS건설', '(주)한화건설', '한신공영', '(주)건영,삼익주택,라이프주택',
       '계룡건설산업(주)', '한양수자인', '건영종합건설(주)', '청담삼익아파트', '계룡건설(주)', '삼환기업',
       '한신공영(주)', 'KR산업', '동부건설', '고속도로관리공단', '동부건설(주)외1', 'SH공사',
       '남광, 대우', 'sh공사', '청광종합건설(주)', '중앙건설(주)', 'LG건설', '신동아건설(주)',
       '현대산업개발(주)', '(주)청구주택', '삼호건설', '(주)삼호', '한솔건설', '현대,대림',
       '신동아 건설(주)', '임광토건(주)', '롯데건설(주)', '벽산건설', '동부건설(주)', '현대 성우건설',
       '홍익종합건업(주)', '한양주택 (주)', '한진중공업(주)', '성지건설', '남광토건(주)', 'gs건설',
       '신명건설,신원건설,미주건설', 'KCC', '두산, 이수건설', '두산건설', 'TEC건설',
       '현대건설,현대산업개발,한진중공업', '성원건설(주)', '라인건설', '코오롱글로벌(주)', '금호건설',
       '(주)대보건설', '(주)한양', '경남기업', '에스에이치공사', '금호', '(주)서광건설산업',
       '우림건설(주)', '중앙하이츠', '청구', '우림건설', '삼환, 삼익건설

기타/의무/임대/임의=1/2/3/4


array(['임의', '의무', '기타', nan, '임대'], dtype=object)

In [8]:
for column in categorical_columns:
     concat[column]  = concat[column].fillna('NULL')

In [9]:
for column in categorical_columns:
    print(column)
    display(concat[column].unique())

시군구


array(['서울특별시 강남구 개포동', '서울특별시 강남구 논현동', '서울특별시 강남구 대치동', '서울특별시 강남구 도곡동',
       '서울특별시 강남구 삼성동', '서울특별시 강남구 세곡동', '서울특별시 강남구 수서동', '서울특별시 강남구 신사동',
       '서울특별시 강남구 역삼동', '서울특별시 강남구 압구정동', '서울특별시 강남구 일원동',
       '서울특별시 강남구 자곡동', '서울특별시 강남구 청담동', '서울특별시 강동구 강일동', '서울특별시 강남구 율현동',
       '서울특별시 강동구 고덕동', '서울특별시 강동구 길동', '서울특별시 강동구 둔촌동', '서울특별시 강동구 명일동',
       '서울특별시 강동구 상일동', '서울특별시 강동구 성내동', '서울특별시 강동구 암사동', '서울특별시 강동구 천호동',
       '서울특별시 강북구 미아동', '서울특별시 강북구 번동', '서울특별시 강서구 가양동', '서울특별시 강북구 수유동',
       '서울특별시 강북구 우이동', '서울특별시 강서구 내발산동', '서울특별시 강서구 등촌동',
       '서울특별시 강서구 공항동', '서울특별시 강서구 마곡동', '서울특별시 강서구 방화동', '서울특별시 강서구 염창동',
       '서울특별시 강서구 화곡동', '서울특별시 관악구 남현동', '서울특별시 관악구 봉천동', '서울특별시 관악구 신림동',
       '서울특별시 광진구 광장동', '서울특별시 광진구 구의동', '서울특별시 광진구 자양동', '서울특별시 광진구 군자동',
       '서울특별시 광진구 화양동', '서울특별시 구로구 개봉동', '서울특별시 구로구 고척동', '서울특별시 광진구 중곡동',
       '서울특별시 구로구 가리봉동', '서울특별시 구로구 구로동', '서울특별시 구로구 오류동',
       '서울특별시 구로구 천왕동', '서울특별시 금천구 독산동', '서울특별시 구로구 온수동', '서울특별시 구로구 궁동',
 

번지


array(['658-1', '652', '12-2', ..., '253-89', '67-5', '240'], dtype=object)

아파트명


...

array(['개별난방', '지역난방', '기타', 'NULL', '중앙난방'], dtype=object)

k-건설사(시공사)


array(['우성건설', '박연정', '대한주택공사', '현대건설(주)', '동양고속건설주식회사', '대우건설',
       '대림산업(주)', '삼성물산', 'NULL', '삼성물산주식회사', '현대,쌍용,GS', '쌍용건설(주)',
       '현대산업개발', '한보', '삼성중공업', '대성산업', '중앙건설', '세방기업', '태영건설', '신동아',
       '신동아건설', '고려개발(주)', '대우건설,대림산업,태영건설', '한라, 두진', '고려산업개발(주)',
       '삼성건설', '롯데건설', 'GS건설', '(주)한화건설', '한신공영', '(주)건영,삼익주택,라이프주택',
       '계룡건설산업(주)', '한양수자인', '건영종합건설(주)', '청담삼익아파트', '계룡건설(주)', '삼환기업',
       '한신공영(주)', 'KR산업', '동부건설', '고속도로관리공단', '동부건설(주)외1', 'SH공사',
       '남광, 대우', 'sh공사', '청광종합건설(주)', '중앙건설(주)', 'LG건설', '신동아건설(주)',
       '현대산업개발(주)', '(주)청구주택', '삼호건설', '(주)삼호', '한솔건설', '현대,대림',
       '신동아 건설(주)', '임광토건(주)', '롯데건설(주)', '벽산건설', '동부건설(주)', '현대 성우건설',
       '홍익종합건업(주)', '한양주택 (주)', '한진중공업(주)', '성지건설', '남광토건(주)', 'gs건설',
       '신명건설,신원건설,미주건설', 'KCC', '두산, 이수건설', '두산건설', 'TEC건설',
       '현대건설,현대산업개발,한진중공업', '성원건설(주)', '라인건설', '코오롱글로벌(주)', '금호건설',
       '(주)대보건설', '(주)한양', '경남기업', '에스에이치공사', '금호', '(주)서광건설산업',
       '우림건설(주)', '중앙하이츠', '청구', '우림건설', '삼환, 삼

기타/의무/임대/임의=1/2/3/4


array(['임의', '의무', '기타', 'NULL', '임대'], dtype=object)

# 시군구 string 시, 구 , 동으로 분리

In [10]:
concat[['시', '구', '동']] = concat['시군구'].str.split(' ', expand=True)

concat.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,시,구,동
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,1.0,0.0,127.05721,37.476763,2022-11-17 10:19:06,124000.0,0,서울특별시,강남구,개포동


In [11]:
concat.drop(["시군구"],axis=1, inplace=True)

In [12]:
concat.head(1)

,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,등기신청일자,...,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,시,구,동
0,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,NaN,...,1.0,0.0,127.05721,37.476763,2022-11-17 10:19:06,124000.0,0,서울특별시,강남구,개포동


In [13]:
modified_train = concat[concat["is_test"]==0]
modified_test = concat[concat["is_test"]==1]
modified_train.head()
modified_train.drop(["is_test"], axis=1, inplace=True)
modified_test.head()
modified_test.drop(["is_test"], axis=1, inplace=True)

modified_test.to_csv("../modified_data/v04_test.csv")
modified_train.to_csv("../modified_data/v04_train.csv")